In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import csv
import time
import sqlite3
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [4]:
#grab detail
conn = sqlite3.connect('scrap.gudangjaket.db')
cur = conn.cursor()
cur.execute('select a.href, a.name from GUDANGJAKET_MASTER a left join GUDANGJAKET_IMAGES b on a.name=b.name where b.name is null limit 1')
reader = cur.fetchall()
data_images = []
# data_desc = []
# data_weight = []
# data_variant = []
for line in reader:
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    driver = webdriver.Chrome('chromedriver',options=chrome_options)
    driver.implicitly_wait(30)
    driver.maximize_window()
    driver.get(line[0])
    elems = driver.find_elements_by_xpath('//div[@data-testid="PDPImageThumbnail"]')
    counting = 0
    product_image = []
    for elem in elems:
        counting = counting + 1
        elem.click()
        time.sleep(3)
        elems = driver.find_element_by_xpath('//img[@alt="product image"]')
        product_image.append(elems.get_attribute("src"))
        if counting == 5:
            break
    driver.quit()
    time.sleep(2)
    rows = [line[1],product_image[0],product_image[1],product_image[2],product_image[3],product_image[4]]
    data_images.append(rows)

#insert images
df = pd.DataFrame(data_images,columns=['name','images1','images2','images3','images4','images5'])
conn = sqlite3.connect('scrap.gudangjaket.db')
df.to_sql('GUDANGJAKET_IMAGES', conn, if_exists='replace', index = False)

# #insert description
# df = pd.DataFrame(data_desc,columns=['name','images'])
# conn = sqlite3.connect('scrap.gudangjaket.db')
# df.to_sql('GUDANGJAKET_DESC', conn, if_exists='append', index = False)

# #insert weight
# df = pd.DataFrame(data_weight,columns=['name','images'])
# conn = sqlite3.connect('scrap.gudangjaket.db')
# df.to_sql('GUDANGJAKET_WEIGHT', conn, if_exists='append', index = False)

# #insert variant
# df = pd.DataFrame(data_variant,columns=['name','images'])
# conn = sqlite3.connect('scrap.gudangjaket.db')
# df.to_sql('GUDANGJAKET_VARIANT', conn, if_exists='append', index = False)


['JAKET OUTDOOR WATERPROOF HOODIE BISA DILEPAS', 'https://ecs7.tokopedia.net/img/cache/700/product-1/2020/4/18/1618811/1618811_20c23047-55c3-4d44-9e6d-887f89dc32c9_1080_1080.webp', 'https://ecs7.tokopedia.net/img/cache/700/product-1/2019/11/28/1618811/1618811_9bb444f7-9341-4cae-b1e8-94e7b5934c2a_1145_1145.webp', 'https://ecs7.tokopedia.net/img/cache/700/product-1/2020/1/7/1618811/1618811_eed2a8f2-f9ed-4ec2-9327-353b9cf6f878_1080_1080.webp', 'https://ecs7.tokopedia.net/img/cache/700/product-1/2020/5/5/1618811/1618811_65f65b88-ba93-48af-85b0-dc34c083cbbf_1280_1280.webp', 'https://ecs7.tokopedia.net/img/cache/700/product-1/2020/5/5/1618811/1618811_7a5eabcd-833f-463b-a263-947f34542c59_1280_1280.webp']
